In [1]:
!pip install geocoder
!pip install folium

In [2]:
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("All Required Libraries Imported!")

All Required Libraries Imported!


In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data, 'lxml')

In [4]:
column_names=['Postal Code','Borough','Neighborhood']
toronto=pd.DataFrame(columns=column_names)
content=wiki_data.find('div',class_='mw-parser-output')
table=content.table.tbody
postcode=0
borough=0
neighborhood=0
for  tr in table.find_all('tr'):
    i=0
    for td in table.find_all('td'):
        if i == 0:
            postcode=td.text
            i=i+1
        elif i==1:
            borough = td.text
            i=i+1
        elif i==2:
            neighborhood = td.text.strip('\n').replace(']','')
    toronto=toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)
            
         

In [5]:
# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1

In [6]:
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,"Not assigned\n, Not assigned\n, Not assigned\n..."


In [7]:
df.describe()

,Postalcode,Borough,Neighborhood
count,1,1,1
unique,1,1,1
top,M1A\n,Not assigned\n,"Not assigned\n, Not assigned\n, Not assigned\n..."
freq,1,1,1


In [8]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [9]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,"Not assigned\n, Not assigned\n, Not assigned\n..."


In [10]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df_2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [11]:
print(df_2.shape)
df_2.head()

(1, 3)


,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,"Not assigned\n, Not assigned\n, Not assigned\n..."


In [12]:
def get_latilong(postal_code):
    lati_long_coords=None
    while(lati_long_coords is None):
        g=geocoder.arcgis('{},Toronto,Ontario'.format(postal_code))
        lati_long_coords=g.latlng
    return lati_long_coords
get_latilong('M4G')

[43.70909000000006, -79.36409999999995]

In [13]:
postal_code=df['Postalcode']
coords=[get_latilong(postal_code) for postal_code  in postal_code.tolist()]

Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)


In [14]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df_2['Latitude'] = df_coords['Latitude']
df_2['Longitude'] = df_coords['Longitude']

In [15]:
df_2[df_2.Postalcode == 'M5G']

,Postalcode,Borough,Neighborhood,Latitude,Longitude


In [16]:
address = 'Scarborough,Toronto'

geolocator = Nominatim(user_agent='ny-explorer')
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The Geograpical Co-ordinate of Seattle,Washington are {}, {}.'.format(latitude_x, longitude_y))

The Geograpical Co-ordinate of Seattle,Washington are 43.773077, -79.257774.


In [17]:
map_Scarborough = folium.Map(location=[latitude_x, longitude_y], zoom_start=10)

for lat, lng, nei in zip(df_2['Latitude'], df_2['Longitude'], df_2['Neighborhood']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

In [18]:
address = 'Scarborough,Toronto'


geolocator = Nominatim(user_agent='ny-explorer')
location = geolocator.geocode(address)
latitude_n1 = location.latitude
longitude_n1 = location.longitude
print('The Geograpical Co-ordinate of Neighborhood_1 are {}, {}.'.format(latitude_x, longitude_y))

The Geograpical Co-ordinate of Neighborhood_1 are 43.773077, -79.257774.


In [21]:
CLIENT_ID = '2YM33GG1WZOH2O4YBA02BFINKESC5TKT5A1PNNN100EPRDRZ' # Put Your Client Id
CLIENT_SECRET = 'FERQC5VE2K2FMRIH1OGTQDW011TNDH0VWGRUDNFBG1TEVC3K' # Put You Client Secret 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: Hidden')
print('CLIENT_SECRET: Hidden')

Your credentails:
CLIENT_ID: Hidden
CLIENT_SECRET: Hidden


In [22]:
radius = 700 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_n1, 
   longitude_n1, 
    radius, 
   LIMIT)
results = requests.get(url).json()

In [23]:
venues=results['response']['groups'][0]['items']
nearby_venues=json_normalize(venues)
nearby_venues.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.crossStreet',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.labeledLatLngs', 'venue.location.distance',
       'venue.location.postalCode', 'venue.location.cc', 'venue.location.city',
       'venue.location.state', 'venue.location.country',
       'venue.location.formattedAddress', 'venue.categories',
       'venue.photos.count', 'venue.photos.groups',
       'venue.location.neighborhood', 'venue.venuePage.id'],
      dtype='object')

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Disney Store,"[{'id': '4bf58dd8d48988d1f3941735', 'name': 'T...",43.775537,-79.256833
1,SEPHORA,"[{'id': '4bf58dd8d48988d10c951735', 'name': 'C...",43.775017,-79.258109
2,American Eagle Store,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",43.776012,-79.258334
3,Hot Topic,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",43.775450,-79.257929
4,DAVIDsTEA,"[{'id': '4bf58dd8d48988d1dc931735', 'name': 'T...",43.776320,-79.258688


In [26]:
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type,axis=1)
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Store,Clothing Store,43.776012,-79.258334
3,Hot Topic,Clothing Store,43.775450,-79.257929
4,DAVIDsTEA,Tea Room,43.776320,-79.258688
5,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
6,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
7,Tommy Hilfiger,Clothing Store,43.776015,-79.257369
8,St. Louis Bar & Grill,Bar,43.774157,-79.253808
9,Shoppers Drug Mart,Pharmacy,43.773305,-79.251662


In [27]:
a=pd.Series(nearby_venues.categories)
a.value_counts()

Clothing Store            8
Restaurant                5
Coffee Shop               5
Gas Station               2
Pharmacy                  2
Tea Room                  2
Sandwich Place            2
Cosmetics Shop            2
Video Game Store          1
Grocery Store             1
Sporting Goods Shop       1
Shopping Mall             1
Department Store          1
Wings Joint               1
Toy / Game Store          1
Greek Restaurant          1
Bakery                    1
Furniture / Home Store    1
Deli / Bodega             1
Fish & Chips Shop         1
Pizza Place               1
Supermarket               1
American Restaurant       1
Bar                       1
Gym                       1
Chocolate Shop            1
Plaza                     1
Food Court                1
Discount Store            1
Smoothie Shop             1
Italian Restaurant        1
Intersection              1
Electronics Store         1
Bank                      1
Mexican Restaurant        1
Movie Theater       

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # making GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
# Nearby Venues
Scarborough_venues = getNearbyVenues(names=df_2['Neighborhood'],
                                   latitudes=df_2['Latitude'],
                                   longitudes=df_2['Longitude']
                                  )

Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assign

In [30]:
print('There are {} Uniques Categories.'.format(len(Scarborough_venues['Venue Category'].unique())))
Scarborough_venues.groupby('Neighborhood').count().head()

There are 62 Uniques Categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n, Not assigned\n",100,100,100,100,100,100


In [32]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]
Scarborough_grouped = Scarborough_onehot.groupby('Neighborhood').mean().reset_index()
Scarborough_onehot.head(5)

,Yoga Studio,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Beer Bar,Bookstore,Brazilian Restaurant,Burrito Place,Café,Coffee Shop,Comedy Club,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Electronics Store,Food Court,Furniture / Home Store,Gastropub,General Travel,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Monument / Landmark,Movie Theater,Museum,Neighborhood,New American Restaurant,Pizza Place,Plaza,Pub,Restaurant,Roof Deck,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Smoke Shop,Soup Place,Souvlaki Shop,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Train Station,University,Vegetarian / Vegan Restaurant,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Not assigned\n, Not assigned\n, Not assigned\n...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,"Not assigned\n, Not assigned\n, Not assigned\n...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Not assigned\n, Not assigned\n, Not assigned\n...",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Not assigned\n, Not assigned\n, Not assigned\n...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Not assigned\n, Not assigned\n, Not assigned\n...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [33]:
num_top_venues = 5
for hood in Scarborough_grouped['Neighborhood']:
    print("---- "+hood+" ----")
    temp =Scarborough_grouped[Scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not assigned
, Not a

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scarborough_grouped['Neighborhood']

for ind in np.arange(Scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Not assigned\n, Not assigned\n, Not assigned\n...",Coffee Shop,Hotel,Café,Beer Bar,Restaurant,Japanese Restaurant,Cosmetics Shop,Concert Hall,Furniture / Home Store,Bookstore


In [36]:
# Using K-Means to cluster neighborhood into 3 clusters
Scarborough_grouped_clustering = Scarborough_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=3, random_state=0).fit(Scarborough_grouped_clustering)
kmeans.labels_

ValueError: n_samples=1 should be >= n_clusters=3